# AutoGen बेसिक नमूना

इस कोड नमूने में, आप [AutoGen](https://aka.ms/ai-agents/autogen) AI फ्रेमवर्क का उपयोग करके एक बेसिक एजेंट बनाएंगे।

इस नमूने का उद्देश्य आपको उन चरणों को दिखाना है जिन्हें हम बाद में विभिन्न एजेंटिक पैटर्न को लागू करते समय अतिरिक्त कोड नमूनों में उपयोग करेंगे।


## आवश्यक पायथन पैकेज आयात करें


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## क्लाइंट बनाएं

इस उदाहरण में, हम [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) का उपयोग LLM तक पहुंचने के लिए करेंगे।

`model` को `gpt-4o-mini` के रूप में परिभाषित किया गया है। GitHub Models मार्केटप्लेस में उपलब्ध किसी अन्य मॉडल पर स्विच करके विभिन्न परिणाम देखने का प्रयास करें।

एक त्वरित परीक्षण के रूप में, हम एक साधारण प्रॉम्प्ट चलाएंगे - `What is the capital of France`।


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## एजेंट को परिभाषित करना

अब जब हमने `client` सेट कर लिया है और यह सुनिश्चित कर लिया है कि यह काम कर रहा है, तो आइए एक `AssistantAgent` बनाते हैं। प्रत्येक एजेंट को निम्नलिखित सौंपा जा सकता है:  
**name** - एक छोटा नाम जो इसे मल्टी-एजेंट फ्लो में संदर्भित करने में उपयोगी होगा।  
**model_client** - वह क्लाइंट जिसे आपने पिछले चरण में बनाया था।  
**tools** - उपलब्ध टूल्स जिनका उपयोग एजेंट किसी कार्य को पूरा करने के लिए कर सकता है।  
**system_message** - वह मेटाप्रॉम्प्ट जो LLM के कार्य, व्यवहार और टोन को परिभाषित करता है।  

आप सिस्टम संदेश को बदल सकते हैं यह देखने के लिए कि LLM कैसे प्रतिक्रिया करता है। हम `tools` को पाठ #4 में कवर करेंगे।  


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## एजेंट चलाएं

नीचे दी गई फ़ंक्शन एजेंट को चलाएगी। हम एजेंट की स्थिति को नए संदेश के साथ अपडेट करने के लिए `on_message` मेथड का उपयोग करते हैं।

इस मामले में, हम उपयोगकर्ता से आने वाले नए संदेश, जो कि `"Plan me a great sunny vacation"` है, के साथ स्थिति को अपडेट करते हैं।

आप संदेश की सामग्री बदल सकते हैं ताकि देख सकें कि LLM अलग-अलग तरीके से कैसे प्रतिक्रिया देता है।


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
